# Import Resource Metadata

This notebook provides an example for uploading table descriptions to Secoda. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [1]:

import requests

API_KEY = "<api_key>"
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Parse your data

If your data is in CSV format, you could parse it this way and standardize the column names in there to match up with Secoda

In [ ]:
import pandas

database = "prod-data-engineering-real"

dataframe = pandas.read_csv("data/mass_maintenance_column_descriptions.csv")
dataframe["title"] = dataframe["column"]
dataframe["description"] = dataframe["col_desc"]
dataframe

## Upload descriptions to Secoda

In [ ]:
import time

def find_secoda_column(title, table, schema, *args, **kwargs):
    res = session.get(
        build_url(f"/table/columns/?title={title}&parent__title={table}&parent__parent__title={schema}&parent__parent__title={database}")
    )
    results = res.json().get("results")
    if len(results) >= 1:
        return results[0]
    return None

def set_column_description(unique_id, description):
    session.put(
        build_url(f"/table/columns/{unique_id}"),
        json=dict(
            description=description
        )
    )
for index, entry in enumerate(dataframe.to_dict("records")):
    print(f"Processing {index}/{dataframe.shape[0]}")
    time.sleep(10)
    column = find_secoda_column(
        **entry
    )
    if column is not None:
        set_column_description(column.get("id"), entry.get("description", ""))